# Computer Vision for image classification

**Author**: Jonathan TRICARD

**Summary**: using a dataset propose by keras, we build a VGG16 model to predict if the images are apple or fish (class 0 or 1). Then, we try to use the Grad Cam method to explain the choice of the model.

**ExplainDL**: create a ```.jpg``` file for each image in the selected path.

## Import libraries

In [ ]:
import itertools
import os

import cv2
import numpy as np

from readml.explainers.dl.explain_dl import ExplainDL
from readml.logger import ROOT_DIR

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D
from tensorflow.keras.models import Model, Sequential


## Initialize the directories

This step is specific to the image classification. We need to create a structure, to save our image data and use it in our model.

In [ ]:
def initialize_directories_dl(out_path, dir_to_create):
    os.chdir(ROOT_DIR)
    new_root = os.getcwd()
    new_root = "/".join(new_root.split("/")[:-1])
    os.chdir(new_root)
    start = out_path.index("/") + 1
    split = out_path[start:].split("/")
    for elt in split:
        if not os.path.isdir(elt):
            os.makedirs(elt)
            os.chdir(elt)
        else:
            os.chdir(elt)
    os.chdir(ROOT_DIR)

    for elt in dir_to_create:
        if not os.path.isdir(os.path.join(out_path, elt)):
            os.makedirs(os.path.join(out_path, elt))


def create_dir_image():
    dir_to_create = ["data_image", "image"]
    out_path = "../outputs/notebooks/dl"
    initialize_directories_dl(out_path, dir_to_create)


create_dir_image()
data_image_path = os.path.join(ROOT_DIR, "../outputs/notebooks/dl", "data_image")
output_path_image_dir = os.path.join(ROOT_DIR, "../outputs/notebooks/dl", "image")

## Import data

In [ ]:
def create_image_data_rgb():
    (train_images, train_labels), (
        test_images,
        test_labels,
    ) = tf.keras.datasets.cifar100.load_data(label_mode="fine")
    _, width, height, channel = train_images.shape
    # Focus on two labels
    focus = list(itertools.chain(*train_labels))
    focus_0_1 = [
        index for index, value in enumerate(focus) if value == 0 or value == 1
    ][0:10]
    train_images = train_images[focus_0_1]
    train_labels = np.array(
        [elt[0] for idx, elt in enumerate(train_labels) if idx in focus_0_1]
    )
    return train_images, train_labels, width, height, channel

def save_image_data(X_train, name):  # data_image_path
    for idx, img in enumerate(X_train):
        cv2.imwrite(os.path.join(data_image_path, f"{name}_{idx+1}.jpg"), img)

In [ ]:
X_train, y_train, width, height, channel = create_image_data_rgb()
save_image_data(X_train, "cifar")

## Train model

In [ ]:
def simple_model_image_rgb(X_train, y_train, width, height, channel):
    baseModel = VGG16(
        weights="imagenet",
        include_top=False,
        input_tensor=Input(shape=(width, height, channel)),
    )
    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(1, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)
    model.compile(loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(X_train, y_train)
    return model

In [ ]:
model = simple_model_image_rgb(X_train, y_train, width, height, channel)

## Make intelligibility with Grad Cam

In [ ]:
model = model
out_path = output_path_image_dir # the path where you want to save the report
image_dir = data_image_path # the path of the image
size = (width, height) # the size of the pictures
color_mode = "rgb"

In [ ]:
exp = ExplainDL(
        model=model,
        out_path=out_path,
    )

In [ ]:
exp.explain_image(
        image_dir=image_dir,
        size=size,
        color_mode=color_mode,
    )